In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[3].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
3116,Fire Emblem: Shadow Dragon,DS,2008.0,Role-Playing,Nintendo,0.31,0.03,0.27,0.03,0.65
3117,Centipede,PS,1998.0,Shooter,Atari,0.36,0.25,0.00,0.04,0.65
3118,Drakengard,PS2,2003.0,Role-Playing,Square Enix,0.19,0.15,0.26,0.05,0.65
3119,SingStar Pop Hits,PS2,2007.0,Misc,Sony Computer Entertainment,0.00,0.50,0.00,0.15,0.65
3120,Kung-Fu Master,2600,1987.0,Action,Activision,0.60,0.04,0.00,0.01,0.65


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for i in range(20): #nombre de micro-scroll à effectuer
        driver.execute_script("window.scrollBy(0, {})".format(value))
        time.sleep(0.1) #Temps entre chaque scroll

In [10]:
gaming_JVC=gaming_parts[3]
gaming_JVC=gaming_JVC.iloc[235:]

In [12]:
#scrapping part 1
#gaming_JVC=gaming_parts[3]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install()) 
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass
    
    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 
    
    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
            else:
                pass
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'
    
    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'
    
    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


100%|██████████| 803/803 [7:19:19<00:00, 32.83s/it]  


In [13]:
driver.close()

In [14]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
3351,LEGO Jurassic World,3DS,2015.0,Action,Warner Bros. Interactive Entertainment,0.31,0.22,0.03,0.05,0.6,15,(2),14,Editeur(s) / Développeur(s) Traveller's Tales ...
3352,Alice: Madness Returns,PS3,2011.0,Adventure,Electronic Arts,0.22,0.25,0.04,0.09,0.6,16,(227),16.4,Editeur(s) / Développeur(s) Spicy Horse Electr...
3353,LEGO The Hobbit,PS4,2014.0,Action,Warner Bros. Interactive Entertainment,0.12,0.37,0.00,0.10,0.6,14,(38),15.2,Editeur(s) / Développeur(s) TT Games Warner In...
3354,Suikoden,PS,1995.0,Role-Playing,Konami Digital Entertainment,0.21,0.14,0.21,0.04,0.6,17,(45),17.7,Editeur(s) / Développeur(s) Konami\nSortie Fra...
3355,Secret Agent Clank,PS2,2009.0,Platform,Sony Computer Entertainment,0.13,0.18,0.00,0.29,0.6,12,(23),12.4,Editeur(s) / Développeur(s) High Impact Games ...
3356,Rayman Arena,PS2,2001.0,Racing,Ubisoft,0.29,0.23,0.00,0.08,0.6,16,(50),16.6,Editeur(s) / Développeur(s) Ubisoft Milan Ubis...
3357,Avatar: The Game,PSP,2009.0,Action,Ubisoft,0.22,0.24,0.00,0.14,0.6,--,no_data,no_data,Editeur(s) / Développeur(s) Fitness Village\nS...
3358,Jaws Unleashed,PS2,2006.0,Action,THQ,0.50,0.02,0.00,0.08,0.6,12,(36),15.8,Editeur(s) / Développeur(s) Appaloosa Majesco\...
3359,Puyo Puyo Sun,SAT,1997.0,Puzzle,Compile,0.00,0.00,0.60,0.00,0.6,--,(0),--,Editeur(s) / Développeur(s) Compile\nSortie Fr...


In [8]:
gaming_JVC.to_csv('gaming_JVC_4bis.csv')